# GAIA DR2


In [1]:
import numpy as np
import pandas as pd
from astroquery.gaia import Gaia
#Gaia.login(user='rgalvez',password='1624354abSN!#')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
import warnings
warnings.filterwarnings('ignore')

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [2]:
%%time

qry = """
SELECT top 1
max(bnb.source_id)as maximumn, min(bnb.source_id) as minimum
FROM gaiadr2.allwise_best_neighbour as bnb
"""
data = Gaia.launch_job_async(qry).get_results().to_pandas()

largest, smallest = data.astype('long').values[0]

Query finished.
CPU times: user 12.8 ms, sys: 848 µs, total: 13.7 ms
Wall time: 761 ms


In [4]:
data = pd.DataFrame()

for i, k in enumerate(range(smallest, largest, largest//1000)):
    
    if i > 830:
        
        qry = """
        SELECT 
        allwise.source_id,
        allwise.original_ext_source_id,
        w1mpro,
        w1mpro_error,
        w2mpro,
        w2mpro_error,
        w3mpro,
        w3mpro_error,
        w4mpro,
        w4mpro_error,
        cc_flags,
        ext_flag,
        var_flag,
        ph_qual,
        w1mjd_mean,
        w2mjd_mean,
        w3mjd_mean,
        w4mjd_mean,
        w1gmag,
        w1gmag_error,
        w2gmag,
        w2gmag_error,
        w3gmag,
        w3gmag_error,
        w4gmag,
        w4gmag_error,
        l, b, dr2.ra, dr2.dec,
        phot_g_mean_mag, bp_rp, bp_g, g_rp,
        pmdec, pmra,
        parallax,
        parallax_over_error

        FROM gaiadr2.allwise_best_neighbour as allwise
        JOIN gaiadr2.gaia_source as dr2 on allwise.source_id = dr2.source_id
        JOIN gaiadr1.allwise_original_valid as original on original.allwise_oid = allwise.allwise_oid

        WHERE allwise.source_id between {} and {}

        """.format(k, k + largest//1000)
        data2 = Gaia.launch_job_async(qry).get_results().to_pandas()
        data  = pd.concat([data, data2], axis=0)
        print(data.shape[0], )
        print(k / largest * 100)

        if i % 10 == 0:
            data.to_csv('/bigdata1/gaia/dr2_allwise_crossmatch_part_{}.csv'.format(i))
            data = pd.DataFrame()
            print('successfully saved file at i = {}'.format(i))
        
data.to_csv('/bigdata1/gaia/dr2_allwise_crossmatch_part_{}.csv'.format(i))
data = pd.DataFrame()
del data2
print('successfully saved file at i = {}'.format(i))
print(data.shape)

Query finished.
328982
83.10000006210029
Query finished.
553471
83.20000006210029
Query finished.
897433
83.3000000621003
Query finished.
1367558
83.4000000621003
Query finished.
1918595
83.50000006210028
Query finished.
2470827
83.60000006210029
Query finished.
3116874
83.7000000621003
Query finished.
3680618
83.80000006210028
Query finished.
4304499
83.9000000621003
Query finished.
4950449
84.0000000621003
successfully saved file at i = 840
Query finished.
618208
84.10000006210029
Query finished.
1182559
84.2000000621003
Query finished.
1718757
84.3000000621003
Query finished.
2290850
84.40000006210029
Query finished.
2833492
84.50000006210028
Query finished.
3341989
84.6000000621003
Query finished.
3795606
84.70000006210029
Query finished.
4273178
84.8000000621003
Query finished.
4716284
84.9000000621003
Query finished.
5124506
85.00000006210028
successfully saved file at i = 850
Query finished.
525162
85.10000006210029
Query finished.
1114301
85.2000000621003
Query finished.
172282

In [16]:
%%time
qry = """
SELECT TOP 2000000 dr2.random_index,
dr2.ra, 
dr2.dec, 
parallax, 
parallax_over_error, 
pmra, 
pmdec, 
ra_dec_corr, 
ra_parallax_corr, 
ra_pmra_corr, 
ra_pmdec_corr, 
dec_parallax_corr, 
dec_pmra_corr, 
dec_pmdec_corr, 
parallax_pmra_corr, 
parallax_pmdec_corr, 
pmra_pmdec_corr,  
phot_g_mean_mag,  
phot_bp_mean_mag, 
phot_rp_mean_mag, 
bp_rp, 
bp_g, 
g_rp

FROM gaiadr2.gaia_source as dr2

WHERE parallax_over_error > 100
AND parallax is not null
AND pmra is not null 
AND pmdec is not null
AND ra_dec_corr is not null
AND ra_parallax_corr is not null
AND ra_pmra_corr is not null
AND ra_pmdec_corr is not null
AND dec_parallax_corr is not null
AND dec_pmra_corr is not null
AND dec_pmdec_corr is not null
AND parallax_pmra_corr is not null
AND parallax_pmdec_corr is not null
AND pmra_pmdec_corr is not null
AND phot_g_mean_mag is not null  
AND phot_bp_mean_mag is not null
AND phot_rp_mean_mag is not null
AND bp_rp is not null
AND bp_g is not null
AND g_rp is not null

"""
data = Gaia.launch_job_async(qry).get_results().to_pandas()
data.to_csv('/bigdata1/gaia/dr2_clean.csv')
print(data.shape)
del data

Query finished.
(2000000, 23)
CPU times: user 2min 46s, sys: 2.47 s, total: 2min 49s
Wall time: 11min 25s


In [29]:
%%time
qry = """
SELECT TOP 2000000 dr2.random_index,
dr2.ra, 
dr2.dec, 
g_mean_psf_mag, 
r_mean_psf_mag, 
i_mean_psf_mag, 
z_mean_psf_mag, 
y_mean_psf_mag,

g_mean_psf_mag_error, 
r_mean_psf_mag_error, 
i_mean_psf_mag_error, 
z_mean_psf_mag_error, 
y_mean_psf_mag_error,

parallax, 
parallax_over_error, 
pmra, 
pmdec, 
ra_dec_corr, 
ra_parallax_corr, 
ra_pmra_corr, 
ra_pmdec_corr, 
dec_parallax_corr, 
dec_pmra_corr, 
dec_pmdec_corr, 
parallax_pmra_corr, 
parallax_pmdec_corr, 
pmra_pmdec_corr,  
phot_g_mean_mag,  
phot_bp_mean_mag, 
phot_rp_mean_mag, 
bp_rp, 
bp_g, 
g_rp

FROM gaiadr2.panstarrs1_original_valid as ps_ov 
JOIN gaiadr2.panstarrs1_best_neighbour as ps_bn on ps_bn.original_ext_source_id = ps_ov.obj_id
JOIN gaiadr2.gaia_source               as dr2   on dr2.source_id = ps_bn.source_id

WHERE parallax_over_error > 100

AND g_mean_psf_mag is not null
AND r_mean_psf_mag is not null
AND i_mean_psf_mag is not null
AND z_mean_psf_mag is not null
AND y_mean_psf_mag is not null
AND y_mean_psf_mag_error is not null
AND g_mean_psf_mag_error is not null
AND r_mean_psf_mag_error is not null
AND i_mean_psf_mag_error is not null
AND z_mean_psf_mag_error is not null
AND y_mean_psf_mag_error is not null
AND parallax is not null
AND parallax_over_error is not null 
AND pmra is not null 
AND pmdec is not null
AND ra_dec_corr is not null
AND ra_parallax_corr is not null
AND ra_pmra_corr is not null
AND ra_pmdec_corr is not null
AND dec_parallax_corr is not null
AND dec_pmra_corr is not null
AND dec_pmdec_corr is not null
AND parallax_pmra_corr is not null
AND parallax_pmdec_corr is not null
AND pmra_pmdec_corr is not null
AND phot_g_mean_mag is not null  
AND phot_bp_mean_mag is not null
AND phot_rp_mean_mag is not null
AND bp_rp is not null
AND bp_g is not null
AND g_rp is not null

"""
data = Gaia.launch_job_async(qry).get_results().to_pandas()
data.to_csv('/bigdata1/gaia/dr2_panstars_clean.csv')
data.shape

Query finished.
CPU times: user 39.7 s, sys: 592 ms, total: 40.3 s
Wall time: 5min 17s


In [6]:
del data 

In [7]:
%%time
qry = """
SELECT TOP 2000000 dr2.random_index,
dr2.ra, 
dr2.dec, 
g_mean_psf_mag, 
r_mean_psf_mag, 
i_mean_psf_mag, 
z_mean_psf_mag, 
y_mean_psf_mag,

g_mean_psf_mag_error, 
r_mean_psf_mag_error, 
i_mean_psf_mag_error, 
z_mean_psf_mag_error, 
y_mean_psf_mag_error,

parallax, 
parallax_over_error, 
pmra, 
pmdec, 
ra_dec_corr, 
ra_parallax_corr, 
ra_pmra_corr, 
ra_pmdec_corr, 
dec_parallax_corr, 
dec_pmra_corr, 
dec_pmdec_corr, 
parallax_pmra_corr, 
parallax_pmdec_corr, 
pmra_pmdec_corr,  
phot_g_mean_mag,  
phot_bp_mean_mag, 
phot_rp_mean_mag, 
bp_rp, 
bp_g, 
g_rp

FROM gaiadr2.panstarrs1_original_valid as ps_ov 
JOIN gaiadr2.panstarrs1_best_neighbour as ps_bn on ps_bn.original_ext_source_id = ps_ov.obj_id
JOIN gaiadr2.gaia_source               as dr2   on dr2.source_id = ps_bn.source_id

WHERE parallax is not null
AND g_mean_psf_mag is not null
AND r_mean_psf_mag is not null
AND i_mean_psf_mag is not null
AND z_mean_psf_mag is not null
AND y_mean_psf_mag_error is not null
AND g_mean_psf_mag_error is not null
AND r_mean_psf_mag_error is not null
AND i_mean_psf_mag_error is not null
AND z_mean_psf_mag_error is not null
AND y_mean_psf_mag_error is not null
AND parallax is not null
AND parallax_over_error is not null 
AND pmra is not null 
AND pmdec is not null
AND ra_dec_corr is not null
AND ra_parallax_corr is not null
AND ra_pmra_corr is not null
AND ra_pmdec_corr is not null
AND dec_parallax_corr is not null
AND dec_pmra_corr is not null
AND dec_pmdec_corr is not null
AND parallax_pmra_corr is not null
AND parallax_pmdec_corr is not null
AND pmra_pmdec_corr is not null
AND phot_g_mean_mag is not null  
AND phot_bp_mean_mag is not null
AND phot_rp_mean_mag is not null
AND bp_rp is not null
AND bp_g is not null
AND g_rp is not null

"""
data = Gaia.launch_job_async(qry).get_results().to_pandas()
data.to_csv('/bigdata1/gaia/dr2_panstars_unclean.csv')
data.shape

Query finished.
CPU times: user 3min 54s, sys: 3.9 s, total: 3min 58s
Wall time: 31min 32s


In [4]:
del data

In [ ]:
%%time
qry = """
SELECT TOP 2000000 dr2.random_index,
dr2.ra, 
dr2.dec, 
g_mean_psf_mag, 
r_mean_psf_mag, 
i_mean_psf_mag, 
z_mean_psf_mag, 
y_mean_psf_mag,

g_mean_psf_mag_error, 
r_mean_psf_mag_error, 
i_mean_psf_mag_error, 
z_mean_psf_mag_error, 
y_mean_psf_mag_error,

parallax, 
parallax_over_error, 
pmra, 
pmdec, 
ra_dec_corr, 
ra_parallax_corr, 
ra_pmra_corr, 
ra_pmdec_corr, 
dec_parallax_corr, 
dec_pmra_corr, 
dec_pmdec_corr, 
parallax_pmra_corr, 
parallax_pmdec_corr, 
pmra_pmdec_corr,  
phot_g_mean_mag,  
phot_bp_mean_mag, 
phot_rp_mean_mag, 
bp_rp, 
bp_g, 
g_rp

FROM gaiadr2.panstarrs1_original_valid as ps_ov 
JOIN gaiadr2.panstarrs1_best_neighbour as ps_bn on ps_bn.original_ext_source_id = ps_ov.obj_id
JOIN gaiadr2.gaia_source               as dr2   on dr2.source_id = ps_bn.source_id

WHERE parallax is not null
AND g_mean_psf_mag is not null
AND r_mean_psf_mag is not null
AND i_mean_psf_mag is not null
AND z_mean_psf_mag is not null
AND y_mean_psf_mag_error is not null
AND g_mean_psf_mag_error is not null
AND r_mean_psf_mag_error is not null
AND i_mean_psf_mag_error is not null
AND z_mean_psf_mag_error is not null
AND y_mean_psf_mag_error is not null
AND parallax is not null
AND parallax_over_error is not null 
AND pmra is not null 
AND pmdec is not null
AND ra_dec_corr is not null
AND ra_parallax_corr is not null
AND ra_pmra_corr is not null
AND ra_pmdec_corr is not null
AND dec_parallax_corr is not null
AND dec_pmra_corr is not null
AND dec_pmdec_corr is not null
AND parallax_pmra_corr is not null
AND parallax_pmdec_corr is not null
AND pmra_pmdec_corr is not null
AND phot_g_mean_mag is not null  
AND phot_bp_mean_mag is not null
AND phot_rp_mean_mag is not null
AND bp_rp is not null
AND bp_g is not null
AND g_rp is not null

"""
data = Gaia.launch_job_async(qry).get_results().to_pandas()
data.to_csv('/bigdata1/gaia/dr2_panstars_unclean.csv')
data.shape

In [7]:
del data

In [ ]:
%%time
qry = """
SELECT 
dr2.source_id, 
w1mpro,
w1mpro_error,
w2mpro,
w2mpro_error,
w3mpro',
w3mpro_error,
w4mpro,
w4mpro_error,
cc_flags,
ext_flag,
var_flag,
ph_qual,
w1mjd_mean,
w2mjd_mean,
w3mjd_mean,
w4mjd_mean,
w1gmag,
w1gmag_error,
w2gmag,
w2gmag_error,
w3gmag,
w3gmag_error,
w4gmag,
w4gmag_error

FROM gaiadr2.allwise_best_neighbour as allwise
JOIN gaiadr2.gaia_source as dr2 on allwise.source_id = dr2.source_id
JOIN gaiadr1.allwise_original_valid as original on original.allwise_oid = allwise.allwise_oid

"""
data = Gaia.launch_job_async(qry).get_results().to_pandas()
#data.to_csv('/bigdata1/gaia/dr2_allwise.csv')
print(data.shape)

In [26]:
data.head()

,source_id,original_ext_source_id,w1mpro,w1mpro_error,w2mpro,w2mpro_error,w3mpro,w3mpro_error,w4mpro,w4mpro_error,...,l,b,phot_g_mean_mag,bp_rp,bp_g,g_rp,pmdec,pmra,parallax,parallax_over_error
0,3.092381e+11,b'J025958.79+000217.4',15.852,0.046,15.547,0.101,12.791,NaN,9.318,NaN,...,176.914265,-48.879747,19.637150,2.705435,1.453381,1.252054,-0.468559,0.918556,1.578841,3.501472
1,3.435974e+11,b'J025951.33+000236.8',15.911,0.046,16.004,0.154,12.718,NaN,9.364,NaN,...,176.875423,-48.898380,18.148487,1.291325,0.579281,0.712044,-1.592207,6.778646,0.112423,0.571341
2,5.497558e+11,b'J030011.58+000253.7',12.682,0.023,12.701,0.025,12.054,0.300,9.155,NaN,...,176.959363,-48.834396,15.003798,1.270053,0.562131,0.707922,-16.297422,0.776524,1.575741,34.676788
3,8.289295e+11,b'J030005.66+000406.3',14.008,0.026,14.052,0.038,12.657,0.428,8.899,NaN,...,176.911302,-48.838163,16.597059,1.479727,0.687414,0.792313,-10.492354,13.599615,1.150084,8.101219
4,9.277131e+11,b'J030006.39+000454.0',16.238,0.055,16.147,0.173,12.628,NaN,9.237,NaN,...,176.900060,-48.826707,19.404552,1.897663,0.856234,1.041430,NaN,NaN,NaN,NaN


In [12]:
data = data.astype('long')

smallest = data.minimum
largest  = data.maximum

difference = largest - smallest
difference

0    6917528993281577216
dtype: int64

In [2]:
%%time
qry = """
SELECT
source_id,
original_ext_source_id

FROM gaiadr2.allwise_best_neighbour

where ra is between {} and {}
""".format()
data = Gaia.launch_job_async(qry).get_results().to_pandas()
data.to_csv('/bigdata1/gaia/dr2_allwise_crossmatch.csv')
print(data.shape)

500


Exception: 500

In [5]:
data.shape

(3000000, 1)

In [3]:
%%time
qry = """
SELECT dr2.source_id, l, b,
phot_g_mean_mag, bp_rp, bp_g, g_rp

FROM gaiadr2.gaia_source as dr2 
WHERE radial_velocity is not null
"""
data = Gaia.launch_job_async(qry).get_results().to_pandas()
data.to_csv('/bigdata1/gaia/dr2_rv_complement.csv')
print(data.shape)

In [24]:
complement = pd.read_csv('/bigdata1/gaia/dr2_allwise_rv_complement.csv').iloc[:,1:]
complement['source_id'] = complement.source_id.astype('long')

In [5]:
from astropy.io import fits
from astropy.table import Table

In [26]:
data = Table.read('/bigdata1/gaia/radial_velocity_sample/Marchetti_v0.1.fits').to_pandas()
data['source_id'] = data.source_id.astype('long')

In [27]:
data.set_index('source_id', inplace=True)
complement.set_index('source_id', inplace=True)

In [37]:
%%time
data2 = data[data.index.isin(complement.index)]
print(data2.shape)

(1603504, 48)
CPU times: user 3.91 s, sys: 285 ms, total: 4.2 s
Wall time: 1.14 s


In [43]:
complement.tail(1).index == data.tail(1).index

array([False])

In [44]:
data.index.isin(complement.index).sum()

1603504

In [58]:
complement.tail()

,w1mpro,w1mpro_error,w2mpro,w2mpro_error,w3mpro,w3mpro_error,w4mpro,w4mpro_error,w1mjd_mean,w2mjd_mean,w3mjd_mean,w4mjd_mean,l,b,phot_g_mean_mag,bp_rp,bp_g,g_rp
source_id,,,,,,,,,,,,,,,,,,
6917441066711387136,10.813,0.023,10.913,0.020,10.646,0.103,9.039,NaN,55406.820502,55406.820502,55325.293087,55325.293087,48.295688,-28.794962,13.328806,1.355224,0.617907,0.737316
6917449244329111552,11.444,0.022,11.482,0.022,11.261,0.172,8.753,NaN,55411.085046,55411.085046,55325.293018,55325.293018,48.501439,-28.619758,12.858150,0.810725,0.328736,0.481989
6917452650238623744,9.744,0.023,9.796,0.021,9.762,0.054,8.257,NaN,55407.868408,55407.868408,55325.683930,55325.683930,48.760382,-28.929388,11.306193,0.896421,0.379481,0.516940
6917467111393043456,11.311,0.022,11.373,0.022,11.353,0.194,8.907,NaN,55418.382660,55418.382660,55324.841987,55324.841987,48.108465,-28.296211,12.912230,0.935438,0.394830,0.540608
6917523843616561152,9.913,0.022,9.957,0.020,9.853,0.053,8.446,NaN,55415.010766,55411.090799,55325.292960,55325.292960,48.787594,-28.369582,11.117123,0.684642,0.273339,0.411303


In [57]:
data.head()[data.head().index.isin(complement.index)]

,ra,dec,parallax,e_parallax,pmra,e_pmra,pmdec,e_pmdec,vrad,e_vrad,...,UW,el_UW,eu_UW,vR,el_vR,eu_vR,vtot,el_vtot,eu_vtot,P_ub
source_id,,,,,,,,,,,,,,,,,,,,,
2851858288640,45.132144,0.137851,2.618833,0.058603,4.782391,0.092797,18.790212,0.080928,33.840437,1.473904,...,29.875095,1.063727,0.978358,30.237784,1.131376,1.029477,272.751037,0.523357,0.510135,0.0
3332894779520,45.058167,0.127404,1.742291,0.049292,6.399016,0.076351,-12.942712,0.084556,4.572674,1.709269,...,18.332150,0.821607,0.856986,-17.313312,1.121078,1.108854,213.336643,1.085285,0.946576,0.0
7632157690368,45.034337,0.235391,5.692796,0.058828,45.377928,0.090056,-6.494226,0.078755,6.404075,0.190706,...,20.268789,0.243004,0.248679,9.606884,0.236312,0.236575,222.288083,0.289838,0.269930,0.0
9281425163264,45.165008,0.200060,3.750882,0.060761,6.535621,0.083478,-53.396270,0.085442,-26.394058,0.586229,...,59.132725,0.572398,0.580229,-58.909873,0.579762,0.569233,202.306197,0.785855,0.710642,0.0
12545600306304,45.234476,0.318159,4.257022,0.044182,0.037377,0.071930,-48.288673,0.075689,-48.504973,2.373606,...,74.305436,1.896763,2.012927,-71.714251,1.582333,1.569737,221.157714,0.696206,0.665785,0.0


In [14]:
data2.to_csv('/bigdata1/gaia/dr2_allwise_rv_marchetti.csv')

In [12]:
%%time
qry = """
SELECT count(*)

FROM gaiadr2.allwise_best_neighbour    as aw_bn
JOIN gaiadr1.allwise_original_valid    as aw_ov on aw_ov.allwise_oid = aw_bn.allwise_oid
JOIN gaiadr2.gaia_source               as dr2   on dr2.source_id = aw_bn.source_id

WHERE parallax is not null
AND pmra is not null 
AND pmdec is not null
--AND ra_dec_corr is not null
--AND ra_parallax_corr is not null
--AND ra_pmra_corr is not null
--AND ra_pmdec_corr is not null
--AND dec_parallax_corr is not null
--AND dec_pmra_corr is not null
--AND dec_pmdec_corr is not null
--AND parallax_pmra_corr is not null
--AND parallax_pmdec_corr is not null
--AND pmra_pmdec_corr is not null
AND phot_g_mean_mag is not null  
AND phot_bp_mean_mag is not null
AND phot_rp_mean_mag is not null
AND bp_rp is not null
AND bp_g is not null
AND g_rp is not null
AND radial_velocity is not null
"""
data = Gaia.launch_job_async(qry).get_results().to_pandas()
# data.to_csv('dr2_allwise.csv')
# data.shape

Query finished.
CPU times: user 1.07 s, sys: 372 ms, total: 1.44 s
Wall time: 10min 54s


In [ ]:
%%time
qry = """
SELECT TOP 1000000 dr2.random_index, *
FROM gaiadr2.gaia_source as dr2
WHERE dr2.parallax is not null
AND dr2.pmra is not null
AND dr2.pmdec is not null
AND dr2.radial_velocity is not null
AND dr2.parallax_over_error > 100
"""
data = Gaia.launch_job_async(qry).get_results().to_pandas()
data.to_csv('/bigdata1/gaia/radial_velocity_dr2_clean.csv')